# Neighborhoods in Toronto Project
### import library

In [13]:
import pandas as pd
import numpy as np 
import urllib.request
from bs4 import BeautifulSoup

### Retrieve toronto postal code data

open url with urllib and getting transforming data with beautiful soup

In [58]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [59]:
soup = BeautifulSoup(page,"html")

making a fonction for transforming td html content to a clean text

In [60]:
def normalize_text(text):
    return text.find(text=True).replace("\n","").strip()

1. define a dictionnary that will content all data
2. find data from html table
3. add data into the dictionnary depending on criteria

In [63]:
data = {'postal_code': [], 'Borough': [], 'Neighborhood': []}
table=soup.find('table', class_='wikitable sortable')
for row in table.findAll("tr"):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_code = normalize_text(cells[0]) 
        borough = normalize_text(cells[1])
        neigh  = normalize_text(cells[2])
        if borough.lower() != 'not assigned':
            if neigh.lower() == 'not assigned':
                neigh = borough
            if postal_code not in data["postal_code"]:
                data["postal_code"].append(postal_code)
                data["Borough"].append(borough)
                data["Neighborhood"].append(neigh)
            else:
                index = data["postal_code"].index(postal_code)
                data["Neighborhood"][index] = "{},{}".format(data["Neighborhood"][index],neigh)

transform data into dataframe and print the result

In [64]:
df = pd.DataFrame(data['postal_code'], columns=['Postal Code'])
df['Borough'] = data['Borough']
df['Neighboorhood'] = data['Neighborhood']
df.head()

,Postal Code,Borough,Neighboorhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [65]:
df.shape

(103, 3)